# Description
This notebook generates cards displaying contested photos based on reactions from other users so they can be printed for usage in workshops. It depends on the CSV outputted by the convertJsonToCSV notebook. The standard settings outputs A5 PDFS containing 1 cards each using the ReportLab Python package (https://www.reportlab.com/dev/install/open_source_installation/)

Note: The cards are drawn based on pixel position, so when making changes it will take some time to get everything in the proper place

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.utils import ImageReader
from reportlab.lib.colors import HexColor
from reportlab.lib.colors import PCMYKColor, PCMYKColorSep, Color, black, blue, red
import os
import json
import time
import csv
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle

In [ ]:
#ADD list of photos
idlist = ['','' ] #list of photoids you want to convert to cards
csv_file = 'dataoutputCSV.csv' #add location of csv file from the convertJsonToCSV notebook

In [ ]:
#set region differently if you want cards in different folders.
region = 'all'
translation_mapping = {
        "Architecture":"Arkitektur",
        "Urban Environment": "Urbant miljø",
        "Objects": "Objekter",
        "Signs & symbols": "Tegn & symboler",
        "Urban Nature": "Bynatur",
        "Wind & Weather": "Vind og vejr",
        "Culture" : "Kultur",
        "Consumption" :"Forbrug",
        "People /community" :"Mennesker/fællesskaber",
        "Infrastructure" :"Infarstruktur",
        "Memories & associations": "Minder & associationer",
        "Atmosphere":"Stemning",
        "Don't know":"Ved ikke",
        "Other":"Andet"
        }


In [ ]:

with open(csv_file, newline='', encoding='utf8') as csvfile:
    photoList = csv.DictReader(csvfile, delimiter=',')
    totalPictures = sum(1 for row in photoList)
 

In [ ]:
print(totalPictures)

In [ ]:

with open(csv_file, newline='', encoding='utf8') as csvfile:
    photos = csv.DictReader(csvfile, delimiter=',')
    j=0
    currentPhoto = 0
    pdfNr = 0
    for photo in list(photos):
        if photo['photoId'] in idlist:
            #print(photo)
            currentPhoto +=1
            annotation_categories = {
                "Architecture": {'pos': 0, 'neg':0},
                "Urban Environment": {'pos': 0, 'neg':0},
                "Objects": {'pos': 0, 'neg':0},
                "Signs & symbols": {'pos': 0, 'neg':0},
                "Urban Nature": {'pos': 0, 'neg':0},
                "Wind & Weather": {'pos': 0, 'neg':0},
                "Culture": {'pos': 0, 'neg':0},
                "Consumption": {'pos': 0, 'neg':0},
                "People /community": {'pos': 0, 'neg':0},
                "Infrastructure": {'pos': 0, 'neg':0},
                "Memories & associations": {'pos': 0, 'neg':0},
                "Atmosphere": {'pos': 0, 'neg':0},
                "Don't know": {'pos': 0, 'neg':0},
                "Other": {'pos': 0, 'neg':0} 
            }
                
            thumbnail_photo = ImageReader(photo['photo_url_original'])
            pos_slider_answers = photo['slider_pos']
            neg_slider_answers = photo['slider_neg']
            total_answer = int(pos_slider_answers) + int(neg_slider_answers)
            code = photo['uniqueCode']
            for key, value in annotation_categories.items():
                #positive
                annotation_categories[key]['pos'] = photo['pos_react'+key]
                #negative
                annotation_categories[key]['neg'] = photo['neg_react'+key]
            
            print(code)
            sortedPosList = [(k,v['pos']) for k, v in sorted(annotation_categories.items(), key=lambda item: item[1]['pos'], reverse=True)]
            sortedNegList = [(k,v['neg']) for k, v in sorted(annotation_categories.items(), key=lambda item: item[1]['neg'], reverse=True)]
  
            if not os.path.exists("photocards_ws2/"+region):
                os.makedirs("photocards_ws2/"+region)
                
            c = canvas.Canvas("photocards_ws2/"+region+"/"+str(code)+".pdf", pagesize=A4)
            c.setFont('Helvetica', 12)
            width, height = A4
        
            c.setFillColor(HexColor('#E3E3E3'))
            c.rect(0,0,600,850, fill=True, stroke=False)
            c.setFillColor(HexColor('#153725'))
            c.setFont('Helvetica-Bold', 25)
    
            c.drawString(50,790,code)
            c.drawImage(thumbnail_photo, 50, 280, width=495, height=495)
    
            
            totalWidth = 492
            #pos_slider_answers = '8'
            #neg_slider_answers = '2'
            dividerColumn = 3
            if pos_slider_answers == '0' or neg_slider_answers == '0':
                 totalWidth = 495
                 dividerColumn = 0
                 
            steps = totalWidth / total_answer
            greenSize = int(pos_slider_answers) * steps
            redSize = int(neg_slider_answers) * steps
    
            c.setFillColor(HexColor('#48A675'))
            c.rect(50,225,greenSize,28, fill=True, stroke=False)
            
            c.setFillColor(HexColor('#E93B3F'))
            c.rect(50+greenSize+dividerColumn, 225, redSize,28, fill=True, stroke=False)
  
            c.setFont('Helvetica', 14)
            #48A675
            c.setFillColor(HexColor('#48A675'))
            c.drawString(50,207, "For mig "+"("+pos_slider_answers+")")
            
            c.setFillColor(HexColor('#E93B3F'))
            c.drawString(452,207, "Ikke for mig "+"("+neg_slider_answers+")")
            
            
      
        
            drawY = 180 
            tableData = []
            for j in range(0,3):
                rowData = []
                posValue = sortedPosList[j][1]
                negValue = sortedNegList[j][1]
                if posValue != '0':
                    posword = sortedPosList[j][0]
                    posword_translation = translation_mapping[posword]
                    rowData.append(posword_translation + ' ('+posValue+')')
                else:
                    rowData.append(' ')
                    
                if negValue != '0':
                    negword = sortedNegList[j][0]
                    negword_translation = translation_mapping[negword]
                    rowData.append(negword_translation + ' ('+negValue+')')
                else:
                    rowData.append(' ')
                tableData.append(rowData)
                    
                
                
               
            print(tableData)
            
            t=Table(tableData, colWidths=[253, 254], rowHeights=30)
            t.setStyle(TableStyle([
                ('ALIGN', (0, 0), (0, -1), "LEFT"),
                ('FONT',(0,0), (-1, -1), 'Helvetica-Bold'),
                ('FONTSIZE', (0,0), (-1, -1), 18),
                ('TEXTCOLOR',(0,0),(0,-1),HexColor('#48A675')),
                ('TEXTCOLOR',(1,0),(1,-1),HexColor('#E93B3F')),
                ('ALIGN', (1, 0), (1, -1), "RIGHT"),
            ]))
            w, h = t.wrapOn(c, 45, 80)
            t.drawOn(c, 45, 80)
            c.save()